To help you prepare your EEG data for input into a ResNet model, we can go through the steps of loading the .edf files, preprocessing the data, and setting up the format for input into ResNet. Here's an outline of the process:

1. **Load .edf files**: We'll use the `pyEDFlib` library to load the EEG data.
2. **Preprocess the EEG data**: This may involve normalization or other preprocessing specific to EEG signals.
3. **Label encoding**: We'll ensure that the labels (normal or abnormal) are mapped into numerical values.
4. **Reshape data for ResNet**: ResNet typically expects input in a specific shape (e.g., (batch_size, height, width, channels)), so we'll format the EEG data accordingly.

In [ ]:
import pyedflib
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Paths to your 10 .edf files
file_paths = ['file1.edf', 'file2.edf', 'file3.edf', 'file4.edf', 'file5.edf',
              'file6.edf', 'file7.edf', 'file8.edf', 'file9.edf', 'file10.edf']

# Labels for the files: 'normal' or 'abnormal'
labels = ['normal', 'abnormal', 'normal', 'normal', 'abnormal', 
          'abnormal', 'normal', 'abnormal', 'normal', 'abnormal']

# Prepare label encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# Function to load and preprocess each .edf file
def load_and_preprocess_edf(file_path):
    f = pyedflib.EdfReader(file_path)
    num_signals = f.signals_in_file
    signal_data = np.zeros((num_signals, f.getNSamples()[0]))

    for i in range(num_signals):
        signal_data[i, :] = f.readSignal(i)
    
    f.close()

    # Preprocess data (e.g., normalization)
    # Here we simply normalize to 0-1 range as an example
    signal_data = (signal_data - np.min(signal_data)) / (np.max(signal_data) - np.min(signal_data))
    
    return signal_data

# Load and preprocess all the files
eeg_data = []
for path in file_paths:
    eeg_data.append(load_and_preprocess_edf(path))

# Convert the list to a numpy array
eeg_data = np.array(eeg_data)

# Reshape data to fit ResNet input shape (assuming 1D signals, we can reshape to (samples, height, width, channels))
# ResNet often takes 2D images, so you might need to adapt depending on your input size
eeg_data = np.expand_dims(eeg_data, axis=-1)  # Add channel dimension

print("EEG data shape:", eeg_data.shape)
print("Labels shape:", categorical_labels.shape)

### Key Steps:
1. **Loading .edf files**: We use `pyedflib` to load the EEG signals.
2. **Preprocessing**: The data is normalized, but you can modify the preprocessing step based on your needs.
3. **Label Encoding**: The labels (normal or abnormal) are encoded into categorical values for training.
4. **ResNet Input Shape**: We ensure the EEG data is reshaped to the required input dimensions for ResNet (e.g., adding a channel dimension).

Once the data is preprocessed, you can split it into training and test sets and feed it into the ResNet model.

Let me know if you'd like to adjust anything, like the preprocessing or data augmentation!

In [1]:
import os
import numpy as np
import pyedflib

# Path to the folders
normal_folder = "'/Users/User/Library/CloudStorage/Box-Box/SeniorDesignT6/TUH EEG Corpus random files/normal'"
abnormal_folder = "'/Users/User/Library/CloudStorage/Box-Box/SeniorDesignT6/TUH EEG Corpus random files/abnormal'"

# Function to load and extract signals from .edf files
def load_edf_signals(file_path):
    with pyedflib.EdfReader(file_path) as f:
        n_signals = f.signals_in_file
        signal_data = np.zeros((n_signals, f.getNSamples()[0]))
        for i in range(n_signals):
            signal_data[i, :] = f.readSignal(i)
    return signal_data

# Prepare data and labels
def prepare_data(normal_folder, abnormal_folder):
    data = []
    labels = []
    
    # Load normal files
    for file_name in os.listdir(normal_folder):
        if file_name.endswith('.edf'):
            file_path = os.path.join(normal_folder, file_name)
            signal_data = load_edf_signals(file_path)
            data.append(signal_data)
            labels.append(0)  # 0 for normal
    
    # Load abnormal files
    for file_name in os.listdir(abnormal_folder):
        if file_name.endswith('.edf'):
            file_path = os.path.join(abnormal_folder, file_name)
            signal_data = load_edf_signals(file_path)
            data.append(signal_data)
            labels.append(1)  # 1 for abnormal
    
    return np.array(data), np.array(labels)

# Load and prepare the dataset
data, labels = prepare_data(normal_folder, abnormal_folder)

# Reshape data for ResNet (ResNet generally expects input shape of (n_samples, height, width, channels))
# Let's assume the EEG data is multichannel, so reshape accordingly
data = np.expand_dims(data, axis=-1)  # Add an extra dimension for channels if needed

# Now 'data' is ready to be fed into a ResNet model
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

FileNotFoundError: [Errno 2] No such file or directory: "'/Users/User/Library/CloudStorage/Box-Box/SeniorDesignT6/TUH EEG Corpus random files/normal'"